# Parrot Predictions Courses

## Boosting - Wisdom of the Crowd

### Idea of boosting
Let's start with intuitive definition of the concept:
> **Boosting** (*Freud and Shapire, 1996*) - algorithm allowing to fit **many** weak classifiers to **reweighted** versions of the training data. Classify final examples by majority voting.

When using boosting techinque all instance in dataset is assigned a score, that is telling *how difficult to classify* it is. In each following iteration the algorithm pay more attention (assign bigger weights) to instances that were previously wrongly classified.

<img src='../images/boosting.png' alt='boosting' style="width: 500px;"/>

For the first iteration all weight are equal.

Ensemble parameters are optimized in **stagewise way** which means that we are calculating optimal parameters for the next classifier holding fixed what was already calculated. This might sound like a limitation but turns out it's a very resonable way of regularizing the model.

### Weak classifier - why tree?
First what is a weak classifier?
> **Weak Classifier** - algorithm **slightly better** than random guessing.

Every algorithm can be used as a base for boosting techinique, but trees have some nice properties that makes them more suitable candidates.

#### Pro's
- computational scalability,
- handling missing values,
- robust to outliers,
- does not require feature scalling,
- can deal with irrelevant inputs,
- interpretable (if small),
- can handle mixed predictors (quantitive and qualitative)

#### Con's
- can't extract linear combination of features
- small predictive power (high variance)

Boosting techinque can try to reduce the variance by **averaging** many **different** trees (where each one is solving the same problem)

### Common Algorithms (warning MATH INCLUDED)

#### AdaBoost (Adaptive Boosting)
The actual implementation of boosting technique using decision tress. The intuitive recipie is presented below:

**Algorithm**:

Assume that the number of training samples is denoted by $N$, and the number of iterations (created trees) is $M$. Notice that possible class outputs are $Y=\{-1,1\}$

1. Initialize the observation weights $w_i=\frac{1}{N}$ where $i = 1,2, \dots, N$
2. For $m=1$ to $M$:
    - fit a classifier $G_m(x)$ to the training data using weights $w_i$,
    - compute $err_m = \frac{\sum_{i=1}^{N} w_i I (y_i \neq G_m(x))}{\sum_{i=1}^{N}w_i}$,
    - compute $\alpha_m = \log ((1-err_m)/err_m)$,
    - set $w_i \leftarrow w_i \cdot \exp [\alpha_m \cdot I (y_i \neq G_m(x)]$, where $i = 1,2, \dots, N$
3. Output $G_m(x) = sign [\sum_{m=1}^{M} \alpha_m G_m(x)]$

In [204]:
import numpy as np

from sklearn.datasets import make_classification
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

# reproducibility
seed = 104
np.random.seed(seed)

Let's consider binary classification case. Generate 20 dimensional dataset with 1000 samples, where 8 features holding information, 3 are redundant and 2 repeated.

In [48]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=8, n_redundant=3, n_repeated=2, random_state=seed)

Split the dataset into train/test parts

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

## A. comparison
With default parameters

### Decision Tree

In [203]:
decision_tree = DecisionTreeClassifier(random_state=seed)
decision_tree.fit(X_train, y_train)

# make predictions
decision_tree_y_pred  = decision_tree.predict_proba(X_test)

# calculate log loss
decision_tree_logloss = log_loss(y_test, decision_tree_y_pred)

print("== Decision Tree ==")
print("Log loss: {0:.2f}".format(decision_tree_logloss))
print("Number of nodes created: {}".format(decision_tree.tree_.node_count))

== Decision Tree ==
Log loss: 7.60
Number of nodes created: 167


### AdaBoost

In [184]:
adaboost = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=1000,
    random_state=seed)
adaboost.fit(X_train, y_train)

# make predictions
adaboost_y_pred = adaboost.predict_proba(X_test)

# calculate log loss
adaboost_logloss = log_loss(y_test, adaboost_y_pred)

print("== AdaBoost ==")
print("Log loss: {0:.2f}".format(adaboost_logloss))

== AdaBoost ==
Log loss: 0.69


### Gradient Boosted Trees

In [206]:
gbc = GradientBoostingClassifier(
    max_depth=1,
    n_estimators=1000,
    warm_start=True,
    random_state=seed)
gbc.fit(X_train, y_train)

# make predictions
gbc_y_pred = gbc.predict_proba(X_test)

# calculate log loss
gbc_logloss = log_loss(y_test, gbc_y_pred)

print("== Gradient Boosting ==")
print("Log loss: {0:.2f}".format(gbc_logloss))

== Gradient Boosting ==
Log loss: 0.48


### XGBoost

In [210]:
xgb = XGBClassifier(
    n_estimators=1000,
    max_depth=1,
    seed=seed
)
xgb.fit(X_train, y_train)

# make predictions
xgb_y_pred = xgb.predict_proba(X_test)

# calculate log loss
xgb_logloss = log_loss(y_test, xgb_y_pred)

print("== XGBoost ==")
print("Log loss: {0:.2f}".format(xgb_logloss))

== XGBoost ==
Log loss: 0.47
